In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import pandas as pd

In [2]:
df = pd.read_csv("../../Datensatz/data.csv")

In [3]:
# Wandle die Spalten "age" und "emotion" in NumPy-Arrays um
X = torch.from_numpy(df[["age", "emotions"]].values).float()
# Wandle das Zielkolumnen-Array in einen PyTorch-Tensor um
y = torch.from_numpy(df["product"].values).long()

In [4]:
# Erstelle die PyTorch-Datensätze
dataset = torch.utils.data.TensorDataset(X, y)

# Teile den Datensatz in Trainings- und Validierungssätze auf
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Erstelle die Dataloader für das Training und die Validierung
train_dataloader = DataLoader(train_dataset, batch_size=1)
test_dataloader = DataLoader(val_dataset, batch_size=1)


In [5]:
train_dataloader

In [6]:

for X, y in test_dataloader:
    print(f"Shape of X [age,emotions]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [age,emotions]: torch.Size([1, 2])
Shape of y: torch.Size([1]) torch.int64


In [7]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(2, 100),
            nn.ReLU(),
            nn.Linear(100, 100),
            nn.ReLU(),
            nn.Linear(100, 26)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=100, bias=True)
    (3): ReLU()
    (4): Linear(in_features=100, out_features=26, bias=True)
  )
)


In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [9]:
def train(model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs):
  # Set model to training mode
  model.train()
  
  for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}\n-------------------------------")
    # Loop over the training data
    for inputs, labels in train_dataloader:
      # Move the inputs and labels to the device where the model is located
      inputs = inputs.to(device)
      labels = labels.to(device)

      # Zero the gradients of the model's parameters
      optimizer.zero_grad()

      # Forward pass: compute the predicted outputs by passing the inputs through the model
      outputs = model(inputs)

      # Compute the loss by comparing the predicted outputs to the true labels
      loss = criterion(outputs, labels)

      # Backward pass: compute the gradients of the loss with respect to the model's parameters
      loss.backward()

      # Update the model's parameters using the gradients and the optimizer's learning rate
      optimizer.step()
      
    # Validate the model on the validation set at the end of each epoch
    val_loss, val_acc = validate(model, val_dataloader, criterion)
    print(f'Epoch {epoch+1}/{num_epochs}: Validation Loss = {val_loss:.3f}, Validation Accuracy = {val_acc:.3f}')

def validate(model, val_dataloader, criterion):
  # Set model to evaluation mode
  model.eval()

  # Initialize variables to track loss and accuracy
  val_loss = 0.0
  num_correct = 0
  num_samples = 0

  # Turn off gradients for validation, saves memory and computations
  with torch.no_grad():
    # Loop over the validation data
    for inputs, labels in val_dataloader:
      # Move the inputs and labels to the device where the model is located
      inputs = inputs.to(device)
      labels = labels.to(device)

      # Forward pass: compute the predicted outputs by passing the inputs through the model
      outputs = model(inputs)

      # Compute the loss by comparing the predicted outputs to the true labels
      loss = criterion(outputs, labels)
      val_loss += loss.item() * inputs.size(0)

      # Compute the accuracy by comparing the predicted labels to the true labels
      _, predicted = torch.max(outputs.data, 1)
      num_correct += (predicted == labels).sum().item()
      num_samples += labels.size(0)

  # Compute the average loss and accuracy
  val_loss /= num_samples
  val_acc = num_correct / num_samples

  return val_loss, val_acc


In [10]:
train(model,train_dataloader,test_dataloader,loss_fn,optimizer,5)

Epoch 1
-------------------------------


IndexError: Target 80 is out of bounds.

In [11]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [12]:
for batch, (X, y) in enumerate(train_dataloader):
    #print(f"X={X}\ny={y}\n\n")
    X, y = X.to(device), y.to(device)
    print(f"X={X}\ny={y}\nx_shape={X.shape}\n y_shape={y.shape}\nmodel(X)={model(X).shape}\n\n")


X=tensor([[20., 10.]])
y=tensor([80])
x_shape=torch.Size([1, 2])
 y_shape=torch.Size([1])
model(X)=torch.Size([1, 26])


X=tensor([[30., 70.]])
y=tensor([260])
x_shape=torch.Size([1, 2])
 y_shape=torch.Size([1])
model(X)=torch.Size([1, 26])


X=tensor([[20., 30.]])
y=tensor([190])
x_shape=torch.Size([1, 2])
 y_shape=torch.Size([1])
model(X)=torch.Size([1, 26])


X=tensor([[20., 60.]])
y=tensor([160])
x_shape=torch.Size([1, 2])
 y_shape=torch.Size([1])
model(X)=torch.Size([1, 26])


X=tensor([[20., 70.]])
y=tensor([140])
x_shape=torch.Size([1, 2])
 y_shape=torch.Size([1])
model(X)=torch.Size([1, 26])


X=tensor([[20., 60.]])
y=tensor([230])
x_shape=torch.Size([1, 2])
 y_shape=torch.Size([1])
model(X)=torch.Size([1, 26])


X=tensor([[20., 10.]])
y=tensor([80])
x_shape=torch.Size([1, 2])
 y_shape=torch.Size([1])
model(X)=torch.Size([1, 26])


X=tensor([[20., 10.]])
y=tensor([120])
x_shape=torch.Size([1, 2])
 y_shape=torch.Size([1])
model(X)=torch.Size([1, 26])


X=tensor([[30., 10.]])
y=t

In [13]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------


IndexError: Target 80 is out of bounds.

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')